In [1]:
import os
import json
import numpy as np
import pandas as pd
import sqlite3
import functools as ft
import matplotlib.pyplot as plt
%matplotlib inline


<strong> Question: What are the most profit categories? What makes application popular? </strong>

In [ ]:
police = pd.read_excel('BIData.xlsx')


count the number of null values in the data set

In [ ]:
police.isnull().sum()

id                            0
state                         0
stop_date                     0
stop_time                500000
location_raw                  0
county_name               37117
county_fips               37117
fine_grained_location    500000
police_department        500000
driver_gender                 0
driver_age_raw                0
driver_age               500000
driver_race_raw               0
driver_race                   0
violation_raw                 0
violation                     0
search_conducted              0
search_type_raw               0
search_type              483038
contraband_found          16141
stop_outcome                  0
is_arrested                   0
ethnicity                     0
dtype: int64

In [ ]:
police.shape

In [ ]:
# ratings.drop(columns=["Unnamed: 0"], inplace=True)
police.isnull().sum()

id                            0
state                         0
stop_date                     0
stop_time                500000
location_raw                  0
county_name               37117
county_fips               37117
fine_grained_location    500000
police_department        500000
driver_gender                 0
driver_age_raw                0
driver_age               500000
driver_race_raw               0
driver_race                   0
violation_raw                 0
violation                     0
search_conducted              0
search_type_raw               0
search_type              483038
contraband_found          16141
stop_outcome                  0
is_arrested                   0
ethnicity                     0
dtype: int64

In [ ]:
police.shape

(500000, 23)

##  Extracting data from JSON file

In [ ]:
with open("installs.json", "r") as file:
    installs = json.load(file)
installs = pd.DataFrame(installs)
installs

FileNotFoundError: ignored

In [ ]:
# pd.read_json(file)

In [ ]:
installs.isna().sum()

NameError: ignored

##  Extracting data from SQL table

In [ ]:
#connection to the database
conn = sqlite3.connect('reviews.sqlite')
cur = conn.cursor()
cur.execute('SELECT * FROM reviews')

# data = cur.fetchall()
# columns = list(map(lambda x: x[0], cur.description))

In [ ]:
#run a query
QUERY = '''
SELECT *
FROM reviews
'''
pd.read_sql(QUERY, conn)

In [ ]:
#run a query
QUERY = '''
SELECT App_Id, AVG(Sentiment) OVER(PARTITION BY App_Id) as Total
FROM reviews
'''
pd.read_sql(QUERY, conn)

In [ ]:
#run a query
QUERY = '''
SELECT *
FROM reviews
'''
pd.read_sql(QUERY, conn)

In [ ]:
pd.read_sql('SELECT AVG("Sentiment_Polarity") FROM reviews', conn)

># Before Transformation, Design Your Schema

## Remove NULL Values / Imputing Data

In [ ]:
police.isna().sum()

In [ ]:
police.dropna(inplace=True)
police.head()

In [ ]:
ratings.isna().sum()

In [ ]:
ratings.head()

In [ ]:
ratings.drop(columns=["Unnamed: 0"], inplace=True)
ratings

In [ ]:
ratings.drop_duplicates(inplace=True)
ratings

In [ ]:
installs.head()

In [ ]:
errors = installs["App_Id"].str.extract(pat='(\w+)', expand=False) # find rows including words
errors

In [ ]:
# "NOT" = ~
errors[~errors.isna()]

In [ ]:
# installs.drop(errors[~errors.isna()].index, inplace=True)
installs

## Remove Duplicates

In [ ]:
police.duplicated().sum()

In [ ]:
ratings.duplicated().sum()

In [ ]:
ratings.drop_duplicates(inplace=True)
# ratings.drop(columns=["Rating_Count"], inplace=True)
ratings.head()

In [ ]:
installs.duplicated().sum()

In [ ]:
installs.drop_duplicates(inplace=True)
installs

## Handling Dates

In [ ]:
parsed_date = pd.to_datetime('January 1st, 2017')
parsed_date

In [ ]:
parsed_date.month

In [ ]:
parsed_date.year

In [ ]:
parsed_date.second

Sometimes date string are formatted in unexpected ways. For example, in the United States, dates are given with the month first and then the day. That is what pandas expects by default. However, some countries write the date with the day first and then the month. Run the next three examples to see Panda's default behavior and how you can specify the date formatting.

In [ ]:
parsed_date = pd.to_datetime('5/3/2017 5:30')
parsed_date.month

In [ ]:
parsed_date = pd.to_datetime('3/5/2017 5:30', format='%d/%m/%Y %H:%M')
parsed_date.month

In [ ]:
parsed_date = pd.to_datetime('5/3/2017 5:30', format='%m/%d/%Y %H:%M')
parsed_date.month

## Dummy Variables

In this exercise, I'll create dummy variables from the projects data set. The idea is to transform categorical data like this:

| Project ID | Project Category |
|------------|------------------|
| 0          | Energy           |
| 1          | Transportation   |
| 2          | Health           |
| 3          | Employment       |

into new features that look like this:

| Project ID | Energy | Transportation | Health | Employment |
|------------|--------|----------------|--------|------------|
| 0          | 1      | 0              | 0      | 0          |
| 1          | 0      | 1              | 0      | 0          |
| 2          | 0      | 0              | 1      | 0          |
| 3          | 0      | 0              | 0      | 1          |



In [ ]:
police

In [ ]:
pd.get_dummies(apps, columns=["Category"])


### replace() method

With pandas, you can use the replace() method to search for text and replace parts of a string with another string. If you know the exact string you're looking for, the replace() method is straight forward. For example, say you wanted to remove the string '(Trial)' from this data:

| data                     |
|--------------------------|
| '(Trial) Banking'        |
| 'Banking'                |
| 'Farming'                |
| '(Trial) Transportation' |

You could use `df['data'].replace('(Trial'), '')` to replace (Trial) with an empty string.

### regular expressions
What about this data?

| data                                           |
|------------------------------------------------|
| 'Other Industry; Trade and Services?$ab' |
| 'Other Industry; Trade and Services?ceg' |

This type of data is trickier. In this case, there's a pattern where you want to remove a string that starts with an exclamation point and then has an unknown number of characters after it. When you need to match patterns of character, you can use [regular expressions](https://en.wikipedia.org/wiki/Regular_expression).

The replace method can take a regular expression. So
df['data'].replace('?.+', regex=True) where '?.+' means find a set of characters that starts with a question mark is then followed by one or more characters. You can see a [regular expression cheat sheet](https://medium.com/factory-mind/regex-tutorial-a-simple-cheatsheet-by-examples-649dc1c3f285) here.


## Finding Outliers


In [ ]:
# JOIN USING(App_Id)
merged = pd.merge(apps, ratings, on=["App_Id"])
merged

In [ ]:
merged.dtypes

Explore the data set to identify outliers using the Tukey rule.

In [ ]:
# merged.plot('Price',kind='box'); -- check what the f*** ?!?!?!
merged["Price"].plot(kind="box")

Use the Tukey rule to determine what values of the population data are outliers for the year 2016. The Tukey rule finds outliers in one-dimension. The steps are:

* Find the first quartile (ie .25 quantile)
* Find the third quartile (ie .75 quantile)
* Calculate the inter-quartile range (Q3 - Q1)
* Any value that is greater than Q3 + 1.5 * IQR is an outlier
* Any value that is less than Qe - 1.5 * IQR is an outlier

In [ ]:
outliers = merged[['App_Id','Price']] # SELECT

Q1 = outliers['Price'].quantile(0.25)
Q3 = outliers['Price'].quantile(0.75)

IQR = Q3 - Q1

# TUKEY: bandwidth: 1.5 outliers --> far...far...far..
bandwidth = 2.5

max_value = Q3 + bandwidth * IQR
min_value = Q1 - bandwidth * IQR

outliers = outliers[(outliers['Price'] > max_value) | (outliers['Price'] < min_value)]
outliers

## Scaling Data

In this exercise, you'll practice scaling data. Sometimes, you'll see the terms **standardization** and **normalization** used interchangeably when referring to feature scaling. However, these are slightly different operations. Standardization refers to scaling a set of values so that they have a mean of zero and a standard deviation of one. Normalization refers to scaling a set of values so that the range if between zero and one.


In [ ]:
pass

# Load

Map all transformation to your <strong>design</strong> and store it.

In [ ]:
merged_data = pd.merge(pd.merge(apps, ratings, on=["App_Id"]), installs, on=["App_Id"])
merged_data

# What is the BIG problem with the above code ??

In [ ]:
SOURCE-TO-TARGET-MAPPING (STTM)